In [1]:
# !pip install pytorch-tabnet

# 용어 정리
- event
- class
- instance
- observation

In [3]:
import os
from glob import glob
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric

import torch.nn.functional as F
import torch.optim as optim
# from datasets import Dataset
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler # 표준화 패키지 라이브러리
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import pickle
from tqdm import tqdm
import random
from warnings import filterwarnings
import gc
from collections import Counter
import matplotlib.pyplot as plt

import data_integration as di
filterwarnings('ignore')
device = torch.device('cpu')
os.getcwd()

'/Users/louan/Documents/projects/Aiffel/Hackathon/geodata-con/geo-con/models/RT_TabNet'

# Data Load

In [8]:
# 전체 file list 만들기
path = '../../../3W/dataset/' # write the path of your '3W/dataset/'
for event in range(9):
    file_lst = []
    full_path = path + f'{event}/*.csv'
    file_lst.extend(sorted(glob(full_path)))
    globals()['df_event_{}'.format(event)] = di.dt_integration(file_lst)
    globals()['df_event_{}'.format(event)].drop(columns = ['timestamp', 'event_type', 'P-JUS-CKGL', 'T-JUS-CKGL', 'QGL', 'instance_type'], inplace = True)
    globals()['df_event_{}'.format(event)].reset_index(inplace=True, drop = True)

../../../3W/dataset/0/*.csv
../../../3W/dataset/1/*.csv
../../../3W/dataset/2/*.csv
../../../3W/dataset/3/*.csv
../../../3W/dataset/4/*.csv
../../../3W/dataset/5/*.csv
../../../3W/dataset/6/*.csv
../../../3W/dataset/7/*.csv
../../../3W/dataset/8/*.csv


In [10]:
def interplolate_without_class(df, method='linear'):
    '''
    input (df) : 라벨 단위 observation_df
    output (df) : 라벨 단위 observation_df
    기본 보간 방법은 선형 보간
    '''
    columns = df.columns
    columns = columns.drop('class')
    class_observations = df['class']

    df = df[columns].interpolate(method = method)
    df['class'] = class_observations
    return df

In [11]:
threshold = 10
events = 9 # 0부터 8번까지의 이벤트를 따로따로 저장함

before_interpolation_description = []
after_interpolation_description = []
observation_nums_lst = []

for event in tqdm(range(event)):
    #1. 인스턴스 별 칼럼의 결측 비율 조사
    df = globals()['df_event_{}'.format(event)]
    missing_proportion_df = di.missing_data_proportion(df)

    #2. Threshold 이상의 observation만 남기고 나머지 observation 제거
    index = missing_proportion_df[(missing_proportion_df > threshold).sum(axis=1) == 0].index
    # print(index) to be deleted
    df = df[df['id_label'].isin(index)]
    print(f"{event} 라벨의 최종 관측 개수: {len(df)}")

    #3. 라벨 제외한 df 보간
    globals()['df_event_{}'.format(event)] = interplolate_without_class(df, method = 'linear')
    


Index(['WELL-00001_20170201020207_0', 'WELL-00001_20170201070114_0',
       'WELL-00001_20170201120124_0', 'WELL-00001_20170201170311_0',
       'WELL-00001_20170201220228_0', 'WELL-00001_20170202030343_0',
       'WELL-00001_20170202080239_0', 'WELL-00001_20170218010146_0',
       'WELL-00001_20170218060218_0', 'WELL-00001_20170218110218_0',
       ...
       'WELL-00006_20170827030218_0', 'WELL-00006_20170827080114_0',
       'WELL-00006_20170827130321_0', 'WELL-00006_20170827180146_0',
       'WELL-00006_20170827230218_0', 'WELL-00006_20170828040343_0',
       'WELL-00006_20170828090311_0', 'WELL-00006_20170828140031_0',
       'WELL-00007_20170801180000_0', 'WELL-00007_20170801200000_0'],
      dtype='object', length=511)
0 라벨의 최종 관측 개수: 8431490
Index(['DRAWN_00001_1', 'DRAWN_00002_1', 'DRAWN_00003_1', 'DRAWN_00004_1',
       'DRAWN_00005_1', 'DRAWN_00006_1', 'DRAWN_00007_1', 'DRAWN_00008_1',
       'DRAWN_00009_1', 'DRAWN_00010_1',
       ...
       'SIMULATED_00109_1', 'SIMULATED

## Outlier 제거 및 변화율 
- 참고: https://stackoverflow.com/questions/35827863/remove-outliers-in-pandas-dataframe-using-percentiles

In [12]:
# Add delta and remove outlier
def add_delta(data_frame):
    for column in ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']:
        column_pctchg = column+'_pctchg'
        data_frame[column_pctchg] = data_frame.groupby(['id_label'])[column].pct_change()
        data_frame[column_pctchg][data_frame[column_pctchg].isna()] = 0
    return data_frame

def remove_outlier(df):
    target_cols = ['P-PDG', 'P-TPT', 'T-TPT','P-MON-CKP', 'T-JUS-CKP',]
    df = df[df['id_label'].str.startswith('WELL')]
    df = add_delta(df)
    Q1 = df[target_cols].quantile(0.25)
    Q3 = df[target_cols].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df[target_cols] < (Q1 - 1.5 * IQR)) | (df[target_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df

In [13]:
# outlier 제거
for i in range(9):
    print(f'event_type: {i} removing outlier')
    globals()['df_event_{}_out'.format(i)] = remove_outlier(globals()['df_event_{}'.format(i)])

event_num: 0
event_num: 1
event_num: 2
event_num: 3
event_num: 4
event_num: 5
event_num: 6
event_num: 7
event_num: 8


# Train Test split

In [14]:
# extract the list of each data type(WELL, SIMULATED, DRAWN)
instance_dict = {'WELL': [], 'SIMULATED': [], 'DRAWN': []}
for i in range(9):
  instance_lst = list(globals()['df_event_{}'.format(i)]['id_label'].unique())
  for instance in instance_lst:
    if instance.startswith('WELL'):
      instance_dict['WELL'].extend([instance])
    elif instance.startswith('SIMULATED'):
      instance_dict['SIMULATED'].extend([instance])
    elif instance.startswith('DRAWN'):
      instance_dict['DRAWN'].extend([instance])

In [15]:
# check the number of events in 'WELL' data
class_lst = []
for i in instance_dict['WELL']:
  class_lst.append(i.split('_')[-1])
Counter(class_lst)

Counter({'0': 511, '4': 344, '3': 32, '6': 6, '7': 5, '1': 4, '2': 3, '5': 3})

In [16]:
# split train and test instances
def split_train_test_instance(instance_dict, dTypes = ['WELL', 'SIMULATED', 'DRAWN'],train_ratio = 0.6, test_ratio = 0.2):
    total_len = 0
    simul_drawn_lst = []
    for dType in dTypes:
        total_len += len(instance_dict[dType])
        simul_drawn_lst += instance_dict[dType]
    well_len = len(instance_dict['WELL'])

    num_of_test = int(total_len*test_ratio)
    num_of_val = int(total_len*(1-train_ratio-test_ratio))
    num_of_train = total_len - num_of_test - num_of_val

    # extract test_lst first(Only WELL data)
    test_lst = random.sample(instance_dict['WELL'], num_of_test)
    
    # then make whole lst --> shuffle lst --> split train validation set
    rest_well_lst = [i for i in instance_dict['WELL'] if i not in test_lst]
    total_lst = rest_well_lst + simul_drawn_lst
    random.shuffle(total_lst)

    train_lst = total_lst[:num_of_train]
    val_lst = total_lst[num_of_train:]

    return train_lst, val_lst, test_lst

In [17]:
train_ratio = 0.7
test_ratio = 0.15
val_ratio = 1 - train_ratio - test_ratio
dTypes = ['WELL', 'SIMULATED', 'DRAWN']
train_lst, val_lst, test_lst = split_train_test_instance(instance_dict, dTypes = dTypes, train_ratio = train_ratio, test_ratio = test_ratio)

# print(len(train_lst), len(val_lst), len(test_lst), sep = '\n')
# print('total number of lst:', len(instance_dict['WELL'])+len(instance_dict['DRAWN'])+len(instance_dict['SIMULATED']))
# print('total number of lst after removing duplicated:', len(set(train_lst + val_lst + test_lst)))
# print('train:', sorted(set([i.split('_')[-1] for i in train_lst])), 'val:', sorted(set([i.split('_')[-1] for i in val_lst])), 'test:', sorted(set([i.split('_')[-1] for i in test_lst])), sep = '\n')

1158
1155
247
total number of lst: 1652
total number of lst after removing duplicated: 1652
train:
['0', '1', '2', '3', '4', '5', '6', '7', '8']
val:
['0', '1', '2', '3', '4', '5', '6', '7', '8']
test:
['0', '3', '4', '5', '7']


In [18]:
# 참고: https://rfriend.tistory.com/590
def make_empty_df(pca_n_components):
    pca_lst = ['pca_1', 'pca_2','pca_3', 'pca_4', 'pca_5']
    columns = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
               'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
               'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
               'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
               'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
               'class',]
    columns.extend(pca_lst[:pca_n_components])
    data_frame = pd.DataFrame(columns=columns)
    return data_frame


def apply_window(data_frame, window_size = 5000, overlap_ratio = 0.2, pca_n_components = 2):
    if pca_n_components:
        pca = PCA(n_components=pca_n_components)
    
    new_df = make_empty_df(pca_n_components)
    for n in tqdm(range(int(len(data_frame)/(window_size*(1-overlap_ratio))))):
        try:
            if int(window_size*(n+1) - window_size*overlap_ratio*n) <= len(data_frame):
                # print(window_size*n*(1-overlap_ratio),window_size*(n+1) - window_size*overlap_ratio*n)
                sliced_df = data_frame[int(window_size*n*(1-overlap_ratio)):int(window_size*(n+1) - window_size*overlap_ratio*n)]
                pca_data = pca.fit_transform(sliced_df.drop( ['id_label','class'], axis=1))
                if any(sliced_df['class'] != 0):
                    class_label =  int(sliced_df['class'].value_counts().index[sliced_df['class'].value_counts().index < 10][0]) # 기본 모드
                    
                    # class_label = 1
                else:
                    # continue
                    class_label = 0
                new_df = pd.concat([new_df,pd.DataFrame({'P-PDG':[sliced_df['P-PDG'].mean()],
                                                        'P-PDG_std':[sliced_df['P-PDG'].std()],
                                                        'P-PDG_pctchg_mean':[sliced_df['P-PDG_pctchg'].mean()],
                                                        'P-PDG_pctchg_std':[sliced_df['P-PDG_pctchg'].std()],
                                                        'P-TPT':[sliced_df['P-TPT'].mean()],
                                                        'P-TPT_std':[sliced_df['P-TPT'].std()],
                                                        'P-TPT_pctchg_mean':[sliced_df['P-TPT_pctchg'].mean()],
                                                        'P-TPT_pctchg_std':[sliced_df['P-TPT_pctchg'].std()],
                                                        'T-TPT':[sliced_df['T-TPT'].mean()],
                                                        'T-TPT_std':[sliced_df['T-TPT'].std()],
                                                        'T-TPT_pctchg_mean':[sliced_df['T-TPT_pctchg'].mean()],
                                                        'T-TPT_pctchg_std':[sliced_df['T-TPT_pctchg'].std()],
                                                        'P-MON-CKP':[sliced_df['P-MON-CKP'].mean()],
                                                        'P-MON-CKP_std':[sliced_df['P-MON-CKP'].std()],
                                                        'P-MON-CKP_pctchg_mean':[sliced_df['P-MON-CKP_pctchg'].mean()],
                                                        'P-MON-CKP_pctchg_std':[sliced_df['P-MON-CKP_pctchg'].std()],
                                                        'T-JUS-CKP':[sliced_df['T-JUS-CKP'].mean()],
                                                        'T-JUS-CKP_std':[sliced_df['T-JUS-CKP'].std()],
                                                        'T-JUS-CKP_pctchg_mean':[sliced_df['T-JUS-CKP_pctchg'].mean()],
                                                        'T-JUS-CKP_pctchg_std':[sliced_df['T-JUS-CKP_pctchg'].std()],
                                                        'class': class_label,
                                                        'pca_1':[pca_data[:,0].mean()],
                                                        'pca_2':[pca_data[:,1].mean()],
                                                        #'pca_3':[pca_data.iloc[:,2].mean()],
                                                        })])

        except:
            continue
    return new_df

def build_dataset(data_frame, train_lst, val_lst, test_lst, window_size = 5000, overlap_ratio = 0.2, pca_n_components = 2):
  gc.collect()
  train_data_frame = data_frame[data_frame['id_label'].isin(train_lst)]
  val_data_frame = data_frame[data_frame['id_label'].isin(val_lst)]
  test_data_frame = data_frame[data_frame['id_label'].isin(test_lst)]

#   train_data_frame = remove_outlier(train_data_frame)

  train_df = apply_window(train_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  val_df = apply_window(val_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  test_df = apply_window(test_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  return train_df, val_df, test_df

In [19]:
# applying window
window_size = 1200
window_size_short = window_size
window_size_long =  window_size
pca_n_components = 2
overlap_ratio = 0.95

df_event_0_window_train, df_event_0_window_val, df_event_0_window_test = build_dataset(df_event_0_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_1_window_train, df_event_1_window_val, df_event_1_window_test = build_dataset(df_event_1_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_2_window_train, df_event_2_window_val, df_event_2_window_test = build_dataset(df_event_2_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_3_window_train, df_event_3_window_val, df_event_3_window_test = build_dataset(df_event_3_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_4_window_train, df_event_4_window_val, df_event_4_window_test = build_dataset(df_event_4_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_5_window_train, df_event_5_window_val, df_event_5_window_test = build_dataset(df_event_5_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_6_window_train, df_event_6_window_val, df_event_6_window_test = build_dataset(df_event_6_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_7_window_train, df_event_7_window_val, df_event_7_window_test = build_dataset(df_event_7_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_8_window_train, df_event_8_window_val, df_event_8_window_test = build_dataset(df_event_8_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
gc.collect()


100%|██████████| 3244/3244 [00:19<00:00, 169.05it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 365/365 [00:02<00:00, 139.42it/s]
0it [00:00, ?it/s]
100%|██████████| 791/791 [00:05<00:00, 147.43it/s]
0it [00:00, ?it/s]
100%|██████████| 4512/4512 [00:20<00:00, 215.52it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


0

In [41]:
# Downsampling train dataset
merged_df_train =  pd.concat([df_event_0_window_train.sample(int(len(df_event_0_window_train)*0.2)),
                              df_event_1_window_train,
                              df_event_2_window_train,
                              df_event_3_window_train.sample(int(len(df_event_3_window_train)*0.5)),
                              df_event_4_window_train.sample(int(len(df_event_4_window_train)*0.2)),
                              df_event_5_window_train,
                              df_event_6_window_train,
                              df_event_7_window_train,
                              df_event_8_window_train])

merged_df_val =  pd.concat([df_event_0_window_val,
                            df_event_1_window_val,
                            df_event_2_window_val,
                            df_event_3_window_val,
                            df_event_4_window_val,
                            df_event_5_window_val,
                            df_event_6_window_val,
                            df_event_7_window_val,
                            df_event_8_window_val])

merged_df_test =  pd.concat([df_event_0_window_test,
                             df_event_1_window_test,
                             df_event_2_window_test,
                             df_event_3_window_test,
                             df_event_4_window_test,
                             df_event_5_window_test,
                             df_event_6_window_test,
                             df_event_7_window_test,
                             df_event_8_window_test])


In [42]:
# print(merged_df_train['class'].value_counts())
# print(merged_df_val['class'].value_counts())
# print(merged_df_test['class'].value_counts())

0    17633
4     4428
3     2943
6       96
5       70
7       64
2       58
Name: class, dtype: int64
0    66164
4    22055
3     4758
6      218
2       64
7       59
5       30
Name: class, dtype: int64
0    21410
4     9310
3     3799
5       30
Name: class, dtype: int64


In [43]:
# P-PDG_pctchg_mean = -inf, 혹은 inf인 row 제외: train(14개), val(10개), test(13)
merged_df_train = merged_df_train[~merged_df_train['P-PDG_pctchg_std'].isna()]
merged_df_val = merged_df_val[~merged_df_val['P-PDG_pctchg_std'].isna()]
merged_df_test = merged_df_test[~merged_df_test['P-PDG_pctchg_std'].isna()]

In [44]:
features = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
            'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
            'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
            'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
            'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
            'pca_1', 'pca_2']
X_train_pre, y_train = merged_df_train.loc[:,features], merged_df_train['class']
X_train_pre

,P-PDG,P-PDG_std,P-PDG_pctchg_mean,P-PDG_pctchg_std,P-TPT,P-TPT_std,P-TPT_pctchg_mean,P-TPT_pctchg_std,T-TPT,T-TPT_std,...,P-MON-CKP,P-MON-CKP_std,P-MON-CKP_pctchg_mean,P-MON-CKP_pctchg_std,T-JUS-CKP,T-JUS-CKP_std,T-JUS-CKP_pctchg_mean,T-JUS-CKP_pctchg_std,pca_1,pca_2
0,0.000000e+00,0.000000e+00,0.0,0.0,8.513980e+06,44250.598242,0.000010,0.000022,118.649417,0.074481,...,1.705502e+06,197762.811138,-1.069346e-05,0.008611,73.865260,0.534615,1.553172e-05,0.000364,3.104409e-11,5.277495e-11
0,0.000000e+00,0.000000e+00,0.0,0.0,8.251894e+06,11768.755682,-0.000004,0.000025,117.081695,0.032046,...,1.738947e+06,235685.005274,2.397799e-04,0.009897,74.262885,0.409763,-9.940235e-06,0.000261,9.313226e-12,4.470348e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,8.210455e+06,16480.355557,0.000005,0.000002,117.338862,0.030220,...,1.510929e+06,231516.942956,2.233274e-04,0.010244,74.310319,0.333256,5.990106e-07,0.000210,-7.761021e-12,-4.610047e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,8.139513e+06,13153.825435,-0.000002,0.000014,116.946400,0.021037,...,1.712754e+06,242015.276350,-2.613143e-04,0.009960,73.995591,0.506100,-3.963848e-07,0.000346,7.605801e-11,1.358179e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,8.520466e+06,20623.757181,0.000005,0.000022,117.630498,0.020783,...,1.500295e+06,239900.723194,9.003129e-05,0.010342,76.527279,0.470989,-2.489537e-06,0.000294,1.055499e-10,-8.436230e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000,0.000000,118.142814,0.010842,...,1.107130e+07,4943.190623,-2.452558e-07,0.000013,69.341553,0.059850,1.089206e-06,0.000003,3.094850e+26,5.820766e-13
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000,0.000000,118.141607,0.010670,...,1.107156e+07,4883.483012,9.712145e-07,0.000013,69.345515,0.059899,4.368168e-07,0.000006,3.094850e+26,-1.940255e-13
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000,0.000000,118.140617,0.011122,...,1.107207e+07,4658.242472,5.985480e-07,0.000013,69.346796,0.059158,1.142284e-06,0.000011,3.094850e+26,1.940255e-13
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000,0.000000,118.140404,0.011407,...,1.107242e+07,4581.638458,4.441322e-07,0.000013,69.355407,0.059855,2.365953e-06,0.000013,3.094850e+26,-1.940255e-13


In [45]:
# normalize
# Pandas automatically applies colomn-wise function in the code above.
# merged_new_df = (merged_df - merged_df.mean())/merged_df.std()
features = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
            'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
            'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
            'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
            'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
            'pca_1', 'pca_2']

X_train_pre, y_train = merged_df_train.loc[:,features], merged_df_train['class']
X_val_pre, y_val = merged_df_val.loc[:,features], merged_df_val['class']
X_test_pre, y_test = merged_df_test.loc[:,features], merged_df_test['class']

# 각 칼럼의 std는 정규화에서 제외
columns = ['P-PDG', 'P-PDG_pctchg_mean','P-TPT','P-TPT_pctchg_mean', 'T-TPT','T-TPT_pctchg_mean', 'P-MON-CKP','P-MON-CKP_pctchg_mean', 'T-JUS-CKP','T-JUS-CKP_pctchg_mean',
           'P-PDG_std', 'P-TPT_std', 'T-TPT_std', 'P-MON-CKP_std', 'T-JUS-CKP_std',
           'pca_1', 'pca_2']

X_train_pre.loc[:,columns] = (X_train_pre.loc[:,columns] - X_train_pre.loc[:,columns].mean())/X_train_pre.loc[:,columns].std()
X_val_pre.loc[:,columns] = (X_val_pre.loc[:,columns] - X_val_pre.loc[:,columns].mean())/X_val_pre.loc[:,columns].std()
X_test_pre.loc[:,columns] = (X_test_pre.loc[:,columns] - X_test_pre.loc[:,columns].mean())/X_test_pre.loc[:,columns].std()

X_train = X_train_pre
X_val = X_val_pre
X_test = X_test_pre

In [46]:
# training
X_train = X_train[features].values
y_train = y_train.values.astype(int)

X_val = X_val[features].values
y_val = y_val.values.astype(int)

X_test = X_test[features].values
y_test = y_test.values.astype(int)

In [47]:
# options
optimizer_fn = torch.optim.AdamW
optimizer_params = dict(lr=0.0001)
batch_size = 64
patience = 10

In [48]:
print(f'{optimizer_fn.__name__}, {train_ratio}, {val_ratio: .2f}, {test_ratio}, long = {window_size_long} short = {window_size_short}, {overlap_ratio}, lr = {optimizer_params}, batch_size = {batch_size}, no_weigth_loss, patience = {patience}, val_acc = ?, test_acc = ?, no Norm, no Simul')

weight_for_class = []
total_obs = len(y_train)
# check
for key, value in sorted(Counter(y_train).items()):
    print(f'{key} is in event, {value}')
    weight_for_class.append(1 - (value/total_obs))

max_epochs = 200
weight_for_class = torch.Tensor(weight_for_class).type(torch.float32)
print("loss function weight:", weight_for_class)

clf = TabNetClassifier(
    optimizer_fn=optimizer_fn,
    optimizer_params=optimizer_params,#1e-2
    # cat_emb_dim = [window_size, window_size, window_size, window_size, window_size]
    )  #TabNetRegressor()

result = clf.fit(
    X_train = X_train,
    y_train = y_train,
    eval_set=[(X_val, y_val)],
    eval_metric = ["accuracy", ],
    loss_fn = nn.CrossEntropyLoss(weight_for_class),
    # loss_fn = nn.CrossEntropyLoss(),
    batch_size = batch_size,
    max_epochs = max_epochs,
    drop_last = True,
    patience = patience
)
# AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.001}, batch_size = 64, no_weigth_loss, patience = 10, val_acc = 0.96924, test_acc = 0.954, no Norm, no Simul
# AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.001}, batch_size = 128, no_weigth_loss, patience = 15, val_acc = .883, test_acc = .957, no Norm, no Simul

AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.0001}, batch_size = 64, no_weigth_loss, patience = 10, val_acc = ?, test_acc = ?, no Norm, no Simul
0 is in event, 17633
2 is in event, 58
3 is in event, 2943
4 is in event, 4428
5 is in event, 70
6 is in event, 96
7 is in event, 64
tensor([0.3028, 0.9977, 0.8836, 0.8249, 0.9972, 0.9962, 0.9975])
epoch 0  | loss: 2.5072  | val_0_accuracy: 0.33197 |  0:00:08s
epoch 1  | loss: 2.20898 | val_0_accuracy: 0.438   |  0:00:16s
epoch 2  | loss: 1.96213 | val_0_accuracy: 0.40697 |  0:00:26s
epoch 3  | loss: 1.74451 | val_0_accuracy: 0.40908 |  0:00:35s
epoch 4  | loss: 1.53917 | val_0_accuracy: 0.3577  |  0:00:44s
epoch 5  | loss: 1.39632 | val_0_accuracy: 0.3698  |  0:00:52s
epoch 6  | loss: 1.27662 | val_0_accuracy: 0.37437 |  0:01:00s
epoch 7  | loss: 1.15218 | val_0_accuracy: 0.39563 |  0:01:09s
epoch 8  | loss: 1.08151 | val_0_accuracy: 0.42398 |  0:01:17s
epoch 9  | loss: 1.01916 | val_0_accuracy: 0.45291 |  0:01:25s
e

In [49]:
preds = clf.predict(X_test)
print(accuracy_score(preds, y_test))
print(classification_report(y_test, preds))

0.9426322035370054
              precision    recall  f1-score   support

           0       0.97      0.95      0.96     21410
           3       0.87      0.92      0.89      3799
           4       0.92      0.94      0.93      9310
           5       0.00      0.00      0.00        30
           7       0.00      0.00      0.00         0

    accuracy                           0.94     34549
   macro avg       0.55      0.56      0.56     34549
weighted avg       0.95      0.94      0.94     34549



In [ ]:
# saving_path_name = './tabnet_test_acc_9904'
# clf.save_model(saving_path_name)